In [1]:
!pip install  langchain
!pip install  langchain_openai
!pip install  langchain_neo4j
!pip install  fastapi[all]
!pip install  requests
!pip install  python-dotenv
!pip install  langchain_community
!pip install  jq
!pip install  certainty-estimator
!pip install  transformers
!pip install  simpletransformers
!pip install neo4j_graphrag

  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 11.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successf

In [2]:
import numpy as np
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph
from langchain.text_splitter import CharacterTextSplitter
from certainty_estimator.predict_certainty import CertaintyEstimator
from dotenv import load_dotenv
from neo4j import GraphDatabase
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
import re
import json
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph
from neo4j_graphrag.retrievers import HybridRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
from neo4j import GraphDatabase
from certainty_estimator.predict_certainty import CertaintyEstimator
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

In [ ]:
estimator = CertaintyEstimator("sentence-level", cuda=True)
hedge_model = NERModel(
    "bert",
    "jeniakim/hedgehog",
    use_cuda=False,
    labels=["C", "D", "E", "I", "N"],
)
tokenizer = AutoTokenizer.from_pretrained(
    "finiteautomata/bertweet-base-sentiment-analysis"
)
conviction_model = AutoModelForSequenceClassification.from_pretrained(
    "finiteautomata/bertweet-base-sentiment-analysis"
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [ ]:
def get_hedge(text):

    predictions, _ = hedge_model.predict([text])

    token_predictions = predictions[0]

    uncertainty_labels = {"D", "E", "I", "N"}

    uncertainty_count = sum(
        1
        for token_dict in token_predictions
        if list(token_dict.values())[0] in uncertainty_labels
    )

    total_tokens = len(token_predictions)
    hedge_score = uncertainty_count / total_tokens if total_tokens > 0 else 0

    return hedge_score

In [ ]:
import torch
import torch.nn.functional as F


def get_conviction_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = conviction_model(**inputs)

    logits = outputs.logits

    probs = F.softmax(logits, dim=-1).squeeze()

    conviction_scores = torch.tensor([-1.0, 0.0, 1.0])

    conviction_score = torch.dot(probs, conviction_scores).item()

    return conviction_score

In [6]:
def get_certainity(text):
    return estimator.predict(text)[0]

In [ ]:
def get_everything(text):
    hedge = get_hedge(text)
    conviction_score = get_conviction_score(text)
    certainity = get_certainity(text)

    return hedge, conviction_score, certainity

In [8]:
review = "This paper considers a special deep learning model and shows that in expectation, there is only one unique local minimizer. As a result, a gradient descent algorithm converges to the unique solution. This works address a conjecture proposed by Tian (2017). While it is clearly written, my main concern is whether this model is significant enough. The assumptions K=2 and v1=v2=1 reduces the difficulty of the analysis, but it makes the model considerably simpler than any practical setting."
rating = "6: Marginally above acceptance threshold"
rating_score = 6
confidence = "2: The reviewer is willing to defend the evaluation, but it is quite likely that the reviewer did not understand central parts of the paper"
confidence_score = 2

In [9]:
print(get_everything(review))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

(0.01282051282051282, 0.41252410411834717, 4.7804375)


In [ ]:
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
OPENAI_API_KEY = ""

In [11]:
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [12]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
embeddings = OpenAIEmbeddings()

In [ ]:
retriever = HybridRetriever(
    driver=driver,
    vector_index_name="reviewEmbeddings",
    fulltext_index_name="reviewFullText",
    embedder=embeddings,
    return_properties=[
        "review",
        "certainty",
        "conviction",
        "hedge",
        "id",
        "confidence",
        "rating",
    ],
)

llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})
rag = GraphRAG(retriever=retriever, llm=llm)

In [ ]:
from typing import List
from pydantic import BaseModel


class Review(BaseModel):
    review: str
    rating: str
    rating_score: int
    confidence: str
    confidence_score: int

In [ ]:
rev1 = Review(
    review=review,
    rating=rating,
    rating_score=rating_score,
    confidence=confidence,
    confidence_score=confidence_score,
)

In [ ]:
def compute_consistency(conviction, certainty, rating, confidence, hedge):
    """
    Compute the Consistency score based on given parameters.

    Parameters:
    conviction (float): Conviction (-1 to 1)
    certainty (float): Certainty (3.3165 to 4.9329)
    rating (float): Rating (6 to 9)
    confidence (float): Confidence (2 to 5)
    hedge (float): Hedge (0 to 0.0734)

    Returns:
    float: Consistency score (1 to 10)
    """
    conviction = max(-1, min(1, conviction))
    certainty = max(3.3165, min(4.9329, certainty))
    rating = max(6, min(9, rating))
    confidence = max(2, min(5, confidence))
    hedge = max(0, min(0.0734, hedge))

    consistency = 1 + 9 * (
        0.4 * ((certainty - 3.3165) / 1.6164)
        + (0.25 + 0.1 * ((rating - 6) / 3) + 0.1 * ((confidence - 2) / 3))
        * ((conviction + 1) / 2)
        + 0.15 * ((rating - 6) / 3)
        + 0.1 * ((confidence - 2) / 3)
        - 0.1 * (hedge / 0.0734)
    )

    return max(1, min(10, consistency))

In [ ]:
compute_consistency(
    get_conviction_score(rev1.review),
    get_certainity(rev1.review),
    rev1.rating_score,
    rev1.confidence_score,
    get_hedge(rev1.review),
)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

5.692329667054569

In [ ]:
def nlp(request: Review):
    try:
        request = (
            "Analyze the similar reviews and their confidence and rating score and give counterfactual reasoning for given review "
            + request.review
            + "with confidence score "
            + request.confidence
            + "and rating score "
            + request.rating
        )
        retriever_result = rag.search(
            query_text=request, return_context=True, retriever_config={"top_k": 5}
        )
        return {"result": retriever_result}
    except Exception as e:
        print(e)

In [39]:
res = nlp(rev1)

In [40]:
print(res)

{'result': RagResultModel(answer="The given review expresses concerns about the significance and practicality of the model being studied, despite acknowledging that the paper is clearly written and addresses a conjecture proposed by Tian (2017). The review has a confidence score of 2, indicating that the reviewer is willing to defend their evaluation but may not have fully understood central parts of the paper. The rating score of 6 suggests that the paper is marginally above the acceptance threshold.\n\nTo provide counterfactual reasoning, we can analyze similar reviews with varying confidence and rating scores:\n\n1. **ICLR_2018_133**: This review has a confidence score of 4 and a rating of 6. The reviewer acknowledges the novelty of the proposed method and its potential for improved performance in certain tasks, but also highlights areas where the method does not perform as expected. The review suggests that addressing these issues could improve the paper's acceptance prospects.\n\n

In [ ]:
def convert_to_dict(obj):
    """
    Recursively converts nested objects to dictionaries.
    Preserves str, int, and float values while converting everything else to dict.

    Args:
        obj: Any Python object

    Returns:
        Converted structure with only dict, list, str, int, float as types
    """
    if isinstance(obj, (str, int, float)):
        return obj

    if isinstance(obj, (list, tuple, set)):
        return [convert_to_dict(item) for item in obj]

    if isinstance(obj, dict):
        return {key: convert_to_dict(value) for key, value in obj.items()}
    if hasattr(obj, "__dict__"):
        return convert_to_dict(obj.__dict__)

    try:
        return {
            key: convert_to_dict(value)
            for key, value in obj.__getattribute__.__self__.__dict__.items()
            if not key.startswith("_")
        }
    except:
        return str(obj)

In [70]:
result = convert_to_dict(res)

In [ ]:
def pretty_print_and_save(dictionary):
    """
    Pretty prints a dictionary and saves it as a JSON file in the specified folder.

    :param dictionary: The dictionary to print and save
    :param folder: The folder where the JSON file should be saved
    :param filename: The name of the JSON file (without extension)
    """
    print(json.dumps(dictionary, indent=4, sort_keys=True))

    with open("/kaggle/working/output.json", "w", encoding="utf-8") as f:
        json.dump(dictionary, f, indent=4, sort_keys=True)

    print(f"Dictionary saved to /kaggle/working/output.json")

In [75]:
pretty_print_and_save(result)

{
    "result": {
        "answer": "The given review expresses concerns about the significance and practicality of the model being studied, despite acknowledging that the paper is clearly written and addresses a conjecture proposed by Tian (2017). The review has a confidence score of 2, indicating that the reviewer is willing to defend their evaluation but may not have fully understood central parts of the paper. The rating score of 6 suggests that the paper is marginally above the acceptance threshold.\n\nTo provide counterfactual reasoning, we can analyze similar reviews with varying confidence and rating scores:\n\n1. **ICLR_2018_133**: This review has a confidence score of 4 and a rating of 6. The reviewer acknowledges the novelty of the proposed method and its potential for improved performance in certain tasks, but also highlights areas where the method does not perform as expected. The review suggests that addressing these issues could improve the paper's acceptance prospects.\

In [76]:
items = result["result"]["retriever_result"]["items"]

total_hedge = total_certainty = total_conviction = total_confidence = total_rating = 0
count = len(items)

for item in items:
    content = eval(item["content"])
    total_hedge += content["hedge"]
    total_certainty += content["certainty"]
    total_conviction += content["conviction"]
    total_confidence += content["confidence"]
    total_rating += content["rating"]

average_hedge = total_hedge / count
average_certainty = total_certainty / count
average_conviction = total_conviction / count
average_confidence = total_confidence / count
average_rating = total_rating / count

print("Averages:")
print(f"Hedge: {average_hedge:.4f}")
print(f"Certainty: {average_certainty:.4f}")
print(f"Conviction: {average_conviction:.4f}")
print(f"Confidence: {average_confidence:.4f}")
print(f"Rating: {average_rating:.4f}")

Averages:
Hedge: 0.0294
Certainty: 4.3442
Conviction: 0.0807
Confidence: 3.8000
Rating: 6.4000


In [ ]:
compute_consistency(
    get_conviction_score(rev1.review),
    get_certainity(rev1.review),
    average_rating,
    average_confidence,
    get_hedge(rev1.review),
)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

6.878462621413623

In [78]:
review = "This paper targets at compressing RNN models to reduce model size, computation cost and enhance model efficiency, without  too much performance drop. This paper proposes to utilize block circulant matrix to speed up the computation as BC matrix multiplication can be computed fast in FFT domain. The idea is straightforward. It seems similar idea has been explored for compressing CNN models  (Ding et al., 2017). But this should be the first work applying BC matrix to compress RNN models. The performance is much better than ESE, in terms of both model computation efficiency and performance in speech applications. The paper is written well. The only issue is it is not clear how BC matrix can be used to compress LSTM, considering LSTM has several different gates with complex interaction. I understand the space is limited but would appreciate if the authors would provide more implementation details for LSTM."
rating = "7: Good paper, accept"
rating_score = 7
confidence = "4: The reviewer is confident but not absolutely certain that the evaluation is correct"
confidence_score = 4

In [ ]:
rev2 = Review(
    review=review,
    rating=rating,
    rating_score=rating_score,
    confidence=confidence,
    confidence_score=confidence_score,
)

In [81]:
print(get_everything(review))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

(0.010416666666666666, 0.6182575225830078, 4.3831954)


In [ ]:
compute_consistency(
    get_conviction_score(rev2.review),
    get_certainity(rev2.review),
    rev2.rating_score,
    rev2.confidence_score,
    get_hedge(rev2.review),
)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

6.8467443887041615

In [82]:
res = nlp(rev2)

In [83]:
result = convert_to_dict(res)
pretty_print_and_save(result)

{
    "result": {
        "answer": "The review in question discusses a paper that focuses on compressing RNN models using block circulant matrices to enhance model efficiency while maintaining performance. The reviewer acknowledges the novelty of applying this technique to RNNs, despite its previous use in CNNs, and appreciates the paper's clarity. However, they express a desire for more details on how the method applies to LSTMs, given their complexity.\n\nCounterfactual reasoning involves considering how changes in certain aspects of the paper or review could affect the confidence and rating scores. Here are some potential counterfactual scenarios:\n\n1. **Increased Detail on LSTM Implementation**: If the paper had provided more comprehensive details on implementing block circulant matrices in LSTMs, the reviewer's confidence might have increased. This additional information could have addressed the reviewer's concerns about the complexity of LSTM gates, potentially leading to a hig

In [84]:
items = result["result"]["retriever_result"]["items"]

total_hedge = total_certainty = total_conviction = total_confidence = total_rating = 0
count = len(items)

for item in items:
    content = eval(item["content"])
    total_hedge += content["hedge"]
    total_certainty += content["certainty"]
    total_conviction += content["conviction"]
    total_confidence += content["confidence"]
    total_rating += content["rating"]

average_hedge = total_hedge / count
average_certainty = total_certainty / count
average_conviction = total_conviction / count
average_confidence = total_confidence / count
average_rating = total_rating / count

print("Averages:")
print(f"Hedge: {average_hedge:.4f}")
print(f"Certainty: {average_certainty:.4f}")
print(f"Conviction: {average_conviction:.4f}")
print(f"Confidence: {average_confidence:.4f}")
print(f"Rating: {average_rating:.4f}")

Averages:
Hedge: 0.0058
Certainty: 4.7290
Conviction: 0.1569
Confidence: 4.0000
Rating: 5.6000


In [ ]:
compute_consistency(
    get_conviction_score(rev2.review),
    get_certainity(rev2.review),
    average_rating,
    average_confidence,
    get_hedge(rev2.review),
)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

6.154005760316711